In [1]:
print("OK all backend is written here")

OK


In [2]:
#import all the libraries
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [26]:
PINECONE_API_KEY=""
PINECONE_API_ENV="" #from index create on pinecone there we get this

In [11]:
#Extract data from the PDf
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader
                           )
    #call the load clas and store it in documents
    documents=loader.load()

    #return this document
    return documents

In [12]:
extracted_data=load_pdf("data/")

In [14]:
#data loaded as document
 #extracted_data

In [17]:
#create text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [18]:
#call the function and see the length of the chunks
text_chunks=text_split(extracted_data)
print("Length of my chunk:",len(text_chunks))

Length of my chunk: 7020


In [19]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [21]:
#call the function
embeddings=download_hugging_face_embeddings()

In [22]:
#print the embedding object
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [23]:
# test the embedding
query_result=embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [25]:
#query_result

In [29]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical--chatbot"


#creating embeddings for each of the text chunks and storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks],  embeddings,index_name=index_name)





AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
# If we already have and index we can load it like this
#perform the semantic search

docsearch=pinecone.from_existing_index(index_name,embeddings)

#
query="What are Allergies"

#searching fro knowledge base and top 3 results it will give
docs=docsearch.similarity_search(query,k=3)


#it will print the ranked result 
print("Result",docs)


In [30]:
# define ,y prompt template
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know tha answer ,just say that you don't know ,don't try to make up an answer.

Context:{context}
Question:{question}

Only return the heplfl answer below and nothing else.
"""

In [31]:
# creating the prompt template
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [33]:
#load my llama model and give the path
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin"
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8
                          })

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1246250385.py, line 2)

In [32]:
#create my qustion answer object amd it will give 2 answer
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs

)

NameError: name 'llm' is not defined

In [ ]:
#ask question 
while True:
    user_input=input(f"Input Prompts:")  #it will take input from the user
    result=qa({"query":user_input})  #ask the question to my llm
    print("Response : ",result["result"])  #llm will give us the response